# Colabocat - Hashcat runtime in Google Colaboratory

In [ ]:
## Dependencies (Mainly for callabcks)
!pip install requests

In [ ]:
HASHCAT_CMD = "{{ HASHCAT_CMD }}"

In [ ]:
!nvidia-smi

In [ ]:
import os, re
import urllib.request

def find_urls(string):
  regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  url = re.findall(regex,string)
  return [x[0] for x in url]

def download_file(url):
  filename = "".join("%02x" % b for b in os.urandom(10))
  fullpath = f"/root/.hashcat/hashes/{filename}"
  urllib.request.urlretrieve(url, fullpath)
  return fullpath

def localize_hashcat_cmd(cmd):
  os.makedirs("/root/.hashcat/hashes/", exist_ok=True)
  for url in find_urls(cmd):
    localized_path = download_file(url)
    cmd = cmd.replace(url, localized_path)
  return cmd    

localized_hashcat_cmd = localize_hashcat_cmd(HASHCAT_CMD)

In [ ]:
callback = """
import requests
import json
import re

def escape_ansi(line):
    ansi_escape =re.compile(r'(\\x9B|\\x1B\[)[0-?]*[ -\/]*[@-~]')
    return ansi_escape.sub('', line)

EMBED_LIMIT = 2048 - 500

url = "{{ DISCORD_WEBHOOK_URL }}"
cmd = \"\"\"{{ HASHCAT_CMD }}\"\"\"

data = {}
data["content"] = ""
data["username"] = "ColaboHash"


results = None
with open("/content/results.txt", "r") as f:
    results = f.readlines()

paginated_results = []
page = ""
for l in results:
    l = escape_ansi(l)
    if len(page) + len(l) >= EMBED_LIMIT:
        paginated_results.append(page)
        page = ""
    page += l
paginated_results.append(page)

for idx, page in enumerate(paginated_results):
    
    data["embeds"] = []

    embed = {}
    embed["title"] = "ColaboHash Results"
    embed["url"] = "https://github.com/apogiatzis/colabohash"
    embed["thumbnail"] = {"url": "https://i.ibb.co/dJ7x7ZH/colabohash400x218.png"}
    embed["description"] = "**Command**: {0}\\n\\n```{1}```".format(cmd, page)
    embed["footer"] = {
        "text": "Page {0}/{1}".format(idx+1, len(paginated_results))
    }

    data["embeds"].append(embed)
    
    result = requests.post(url, data=json.dumps(data), headers={"Content-Type": "application/json"})

    try:
        result.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
"""


with open("/content/callback.py", "w") as f:
    f.write(callback)

In [ ]:
hashcat_script = """
#!/bin/bash

{0} > results.txt

python /content/callback.py

{% if TERMINATE_ON_FINISH %}
jupyter notebook stop 9000
{% endif %}
""".format(localized_hashcat_cmd)

with open("/content/hashcat_script.sh", "w") as f:
    f.write(hashcat_script)

!chmod +x /content/hashcat_script.sh

In [ ]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

In [ ]:
!nohup /content/hashcat_script.sh &

In [ ]:
## Keep NB Alive (12 hours)

!read